In [1]:
import pandas as pd
import os
from zipfile import ZipFile
from bs4 import BeautifulSoup

# Analysis on data dump
This notebook is to learn reading data dumps directly form the server (Rnd2) as opposed to obtain the data through API calls

Recent data dump with normalized lang tags produced at the end of June 2022 for the project Etranslate

In [1]:
#data dump location
data_path="/projects/etranslate-data-dump"

In [2]:
!pwd

/home/pscalia/git/rd-europeana-translate/data_analysis/from_datadump


In [12]:
#listing files in directory
list_data_files=os.listdir(data_path)

In [8]:
2051943

2051943

In [17]:
#locating zip file. Each zip file  contains a collection
file_name = os.path.join(data_path,'2051943.zip')
file_name

'/projects/etranslate-data-dump/2051943.zip'

In [18]:
#reading files that

In [ ]:
# extrecting  files that have not been extracted
with ZipFile(file_name, 'r') as zipObject:
 listOfFileNames = zipObject.namelist()
 for fileName in listOfFileNames:
        if os.path.isfile(fileName):
           # Extract a single file from zip
            print('file already exist')
        else:
            zipObject.extract(fileName)
            print('file extracted')

In [46]:
#open one of the xml unzipped files
with open(listOfFileNames[0], 'r') as f:
	file = f.read() 

In [47]:
#use beaut soup for parsing xml
soup = BeautifulSoup(file, 'xml')

In [ ]:
#finding all description tag
soup.find('description')

In [ ]:
t.text

In [ ]:
# find attributes of title
soup.find()

# Experiment 1: extract all german text

In [ ]:
listOfFileNames

In [ ]:
with ZipFile(file_name, 'r') as zipObject:
 listOfFileNames = zipObject.namelist()
 for fileName in listOfFileNames:
        if os.path.isfile(fileName):
           # Extract a single file from zip
            print('file already exist')
            print(fileName)
        else:
            zipObject.extract(fileName)
            print('file extracted') 

In [7]:
def read_single_collection(listOfFileNames, field):
    lang_tags=[]
    for filename in listOfFileNames:
        with open(filename, 'r') as f:
            file = f.read() 
        soup = BeautifulSoup(file, 'xml')
        t=soup.find(f'{field}')
        lang=t.get('xml:lang')
        if lang:
            lang_tags.append(lang)
    return lang_tags

In [ ]:
list_data_files=os.listdir(data_path) #list of zip files
for zip_file in list_data_files:
    file_name = os.path.join(data_path,  zip_file) #name of zip file
    print(file_name)
    with ZipFile(file_name, 'r') as zipObject:
        listOfFileNames = zipObject.namelist()
        for fileName in listOfFileNames:
                if os.path.isfile(fileName):
                   # Extract a single file from zip
                   print('file exists')
                   os.remove(fileName)  
                else:
                    pass
         
        
                
                #     extracted_file=zipObject.extract(fileName)
                #     print('file extracted')
                #     with open( extracted_file, 'r') as f:
                #         file = f.read() 
                #     soup = BeautifulSoup(file, 'xml')
                #     t=soup.find('description')
                #     print(t.get('xml:lang'))
                #     print('file extracted')

In [38]:
for values in soup.findAll("time"):
    print("{} : {}, {}°".format(values["from"], values.find("symbol")["name"], values.fin

SyntaxError: unexpected EOF while parsing (1312526846.py, line 2)

In [ ]:
file_name = os.path.join(data_path,  '2051943.zip')
file_name

In [61]:
with open('plink__f_1_100475.xml', 'r') as f:
    file = f.read() 
soup = BeautifulSoup(file, 'xml')
t=soup.find('temporal')
#t.get('xml:lang')
t

<dcterms:temporal xml:lang="de">Mai 1952</dcterms:temporal>

In [15]:
file_name = os.path.join(data_path,  '2051943.zip')
file_name
    

'/projects/etranslate-data-dump/2051943.zip'

In [ ]:
file_name = os.path.join(data_path,  list_data_files[2])

In [ ]:
with ZipFile(file_name, 'r') as zipObject:
    listOfFileNames = zipObject.namelist()
 for fileName in listOfFileNames:
        if os.path.isfile(fileName):
           # Extract a single file from zip
            print('file already exist')
        else:
            data=zipObject.extract(fileName)
            print('file extracted')

In [ ]:
#alternative to find keyword in file?
keyword = 'your_keyword'
for file in files:
    if os.path.isfile(os.path.join(your_path, file)):
        f = open(os.path.join(your_path, file),'r')
        for x in f:
            if keyword in x:
                #do what you want
        f.close()